In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
import astra
astra.set_gpu_index((0, 1, 2, 3))

from AdversarialRegularizer_FoV import AdversarialSplitting
from networks_FoV import ResNetL2, ResNetL2_local
from ellipsgen.generate_data_function import get_batch
import numpy as np
import ellipsgen.CCB_class as CT
import ellipsgen.phantom_class as ph
import matplotlib.pyplot as plt

In [2]:
# Parameter choices. Heuristic in the BWGAN paper: Choose GAMMA as average dual norm of clean image
# LMB should be bigger than product of norm times dual norm.

# For s=0.0, this implies GAMMA =1.0
# For s=1.0, have GAMMA = 10.0 as realisitc value

saves_path = '/export/scratch1/home/voxels-gpu0/codesprint_learned_prior/AdvRegSaves/FoV_experiments'
regularizer = AdversarialSplitting(saves_path, NETWORK=ResNetL2_local, BATCH_SIZE=1, gamma=1.0)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.conv3d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.average_pooling3d instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
No save found


In [3]:
def load_pair(i):
    prefix = '../noisy_' + str(IMG_SIZE) + '/'
    fdk = np.load(prefix + 'FDK/dataset' + str(i) + '.npy')
    gt = np.load(prefix + 'GT/dataset' + str(i) + '.npy')
    
    return fdk, gt

IMG_SIZE = 128
def generate_pairs(amount=1):
    num_angles = 360
    noise = ['Poisson', 2 ** 14] # Do not go below 2 ** 8, lower number in the 2nd argument means more noise
    num_vox = IMG_SIZE
    
    voxels = [num_vox, num_vox, num_vox]
    src_rad = 10
    det_rad = 0

    fdks = np.empty((amount, *voxels))
    gts = np.empty_like(fdks)

    for i in range(amount):
        data_obj = ph.phantom(voxels, '22 Ellipses', num_angles, noise, src_rad, det_rad)
        gts[i, ...] =  data_obj.f

        case = CT.CCB_CT(data_obj)
        fdks[i, ...] = case.do_FDK()
        
        if i == 0:
            plt.figure()
            plt.imshow(gts[i, int(IMG_SIZE/2), ...])
            plt.show()
            
            plt.figure()
            plt.imshow(fdks[i, int(IMG_SIZE/2), ...])
            plt.show()
            
    
    return (gts[0,...], fdks[0,...])

In [4]:
def evaluate():
    adv, gt = load_pair(99)
    regularizer.test(groundTruth=gt, adversarial=adv)

    
def train(steps):
    for k in range(steps):
        adv, gt = load_pair(steps%99)
        regularizer.train(groundTruth=gt, adversarial=adv, learning_rate=LEARNING_RATE)
        if k%5 == 0:
            evaluate()        
    regularizer.save()

In [ ]:
LOOPS = 1
STEPS = 100

LEARNING_RATE = 2e-5
for k in range(LOOPS):
    train(STEPS)

LEARNING_RATE = 1e-5
for k in range(LOOPS):
    train(STEPS)

Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Evaluation on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98, 98, 98)
Training on batch of size (1, 98

In [14]:
adv, gt = load_pair(99)
np.sum((adv-gt) ** 2)/ (np.sum(adv ** 2))


0.017810458

In [6]:
adv_grad = regularizer.evaluate(adv)

In [ ]:
step_len = 1
reco = np.copy(adv)
mu = 0.0

for k in range(20):
#    print(np.mean(reco))
#    print(np.mean(regularizer.evaluate(reco)))
    reco = reco - step_len * (regularizer.evaluate(reco) + mu*(reco-adv))
    
    if k%5 == 0:
        plt.imshow(np.maximum(reco[64, ...], 0))
        plt.colorbar()
        plt.show()

NameError: name 'adv' is not defined